# Text Mining with Sklearn



__Author:__ Colin Cambo

__Date:__ 4/18/2017

__About:__ This notebook covers the basics of text mining with sklearn.

__Contents:__

  * [0 Introduction 20 newsgroups Dataset](#0)
    * [0.1 Loading Data](#0.1)
    * [0.2 Simple Exploration](#0.2)
  * [1 Text Mining Basics](#1)
    * [1.1 CountVectorizer](#1.1)
        * [1.1.1 Description](#1.1.1)
        * [1.1.2 Example](#1.1.2)
        * [1.1.3 Parameters](#1.1.3)
          * [1.1.3.1 min_df](#1.1.3.1)
          * [1.1.3.2 max_df](#1.1.3.2)
          * [1.1.3.3 ngram_range](#1.1.3.3)
          * [1.1.3.4 max_features](#1.1.3.4)
    * [1.2 TfidfTransformer](#1.2)
      * [1.2.1 Description](#1.2.1)
      * [1.2.2 Example](#1.2.2)
      * [1.2.3 Parameters](#1.2.3)
          * [1.2.3.1 norm](#1.2.3.1)
          * [1.2.3.2 use_idf](#1.2.3.2)
  * [2 Modeling](#2)
    * [2.1 Multinomial Naive Bayes](#2.1)
      * [2.1.1 Description](#2.1.1)
      * [2.1.2 Example](#2.1.2)
      * [2.1.3 Parameters](#2.1.3)
        * [2.1.3.1 alpha](#2.1.3.1)
    * [2.2 Stochastic Gradient Descent](#2.2)
      * [2.2.1 Description](#2.2.1)
      * [2.2.2 Example](#2.2.2)
      * [2.2.3 Parameters](#2.2.3)
        * [2.2.3.1 alpha](#2.2.3.1)
        * [2.2.3.2 epsilon](#2.2.3.2)
  * [3 Tuning Models](#3)
    * [3.1 Pipelines](#3.1)
      * [3.1.1 Description](#3.1.1)
      * [3.1.2 Example](#3.1.2)
    * [3.2 Parameter Seach](#3.2)
      * [3.2.1 GridSearchCV](#3.2.1)
        * [3.2.1.1 Description](#3.2.1)
        * [3.2.1.2 Example](#3.2.2)
        * [3.2.1.3 Parameters](#3.2.3)
      * [3.2.1 RandomizedSearchCV](#3.2.1)
        * [3.2.1.1 Description](#3.2.1)
        * [3.2.1.2 Example](#3.2.2)
        * [3.2.1.3 Parameters](#3.2.3)
  * [4 Increasing Performance](#4)
    * [4.1 Stemming](#4.1)
      * [4.1.1 Description](#4.1.1)
      * [4.1.2 Example](#4.1.2)
      * [4.1.3 Types](#4.1.3)
        * [2.2.3.1 Porter](#4.1.3.1)
        * [2.2.3.1 Lancaster](#4.1.3.2)
        * [4.1.3.3 Snowball](#4.1.3.3)
    * [4.2 Custom Transformers](#4.2)
      * [4.2.1 ColSelector](#4.2.1)
        
__Requirements__

Run the cell below to pip install all dependencies

In [ ]:
#!pip install -r requirements.txt

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, accuracy_score, log_loss
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier
from ipywidgets import interact
from time import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

<a id='0'></a>
# 0 Introduction 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.

For more information about the data: http://qwone.com/~jason/20Newsgroups/

<a id='0.1'></a>
## 0.1 Loading Data

We will use sklearn to load the data, so we will import the function fetch_20newsgroups from sklearn.datasets

In [2]:
from sklearn.datasets import fetch_20newsgroups

The default parameters will return the entire dataset as a python object. Below we will set data equal to the dataset

In [3]:
data = fetch_20newsgroups()

In [4]:
#prints all objects inside train
dir(data)

['DESCR', 'data', 'description', 'filenames', 'target', 'target_names']

In [ ]:
train.description

Target names are stored in data.target_names, below are all of the target names currently in our dataset. 

In [ ]:
data.target_names

Using Counter we can see just how many observations belong to each of our classes

In [ ]:
from collections import Counter

In [ ]:
Counter(data.target)

Overall the dataset is pretty balanced between the classes, which makes classification easier for us, but there are still too many observations for us to effectively model in a short period, so we will subset the dataset by taking only 5 classes.

To subset the dataset by classes you just have to pass a list of the classes you're interested in to the fetch_20newsgroups function's categories variable. We will also subset our dataset to just get the train set.

In [ ]:
cat = ['rec.sport.baseball', 'sci.electronics', 'talk.politics.misc', 'sci.crypt', 'misc.forsale']

In [ ]:
train = fetch_20newsgroups(subset='train', categories=cat, shuffle=True, random_state=42)

<a id='0.2'></a>
## 0.2 Simple Data Exploration

This lecture is for text mining so our data exploration will be minimal

In [ ]:
len(train.data)

In [ ]:
len(train.target)

In [ ]:
train.target_names

In [ ]:
Counter(train.target)

In [ ]:
df = pd.DataFrame({'Text':train.data, 'Target':train.target})

In [ ]:
df.head()

In [ ]:
print(df.iloc[1]['Text'])

This isn't your usual text data, so we will see if this still follows Zipf's law

__[Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law):__  Zipf's law states that given some corpus of natural language utterances, the frequency of any word is inversely proportional to its rank in the frequency table. Thus the most frequent word will occur approximately twice as often as the second most frequent word, three times as often as the third most frequent word, etc.: the rank-frequency distribution is an inverse relation.

In [ ]:
from functools import reduce
import re
token_regex = re.compile(r'\b\w\w+\b')

#Here we're just combining all tokens into one massive string
all_msgs = reduce((lambda x, y: x + y), map(lambda x: token_regex.findall(x.lower()), df['Text'].tolist()))

In [ ]:
count = Counter(all_msgs)

In [ ]:
#Sorting counts
word_counts = sorted(list(count.items()), key=lambda x:x[1], reverse=True)

In [ ]:
#Displaying first 10
word_counts[:10]

In [ ]:
subset_word_counts = word_counts[:1000]
plt.figure(figsize=(8,8))
plt.scatter(x=range(len(subset_word_counts)), y=[w[1] for w in subset_word_counts])
plt.title('Count for Words')
plt.ylabel('Count')
plt.xlabel('Word index')

In [ ]:
subset_word_counts = word_counts[:1000]
plt.figure(figsize=(8,8))
plt.scatter(x=range(len(subset_word_counts)), y=[np.log(w[1]) for w in subset_word_counts])
plt.title('log of Count for Words')
plt.ylabel('log(Count)')
plt.xlabel('Word index')

Yup that's Zipf's law

<a id='1'></a>
# 1 sklearn Text Mining Basics

sklearn has many great tools for text mining, in this section we will cover two of the more important ones

<a id='1.1'></a>
## 1.1 CountVectorizer

In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors. This is typically done with __Bag-of-Words__, and this is where the CountVectorizer class comes in handy. You can read more about the CountVectorizer [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

<a id='1.1.1'></a>
### 1.1.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)__

Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using scipy.sparse.coo_matrix.

<a id='1.1.2'></a>
### 1.1.2 Example

To get a better understanding of the class we will do an example below

In [ ]:
text = ['cat dog dog dog dog dog dog dog', 'cat fish puppy', 'fish cat']

In [ ]:
vect = CountVectorizer()

In [ ]:
#vect.fit(text)
#dtm = vect.transform(text)
dtm = vect.fit_transform(text)

In [ ]:
print(dtm)

In [ ]:
dtm.todense()

In [ ]:
vect.get_feature_names()

In [ ]:
df2 = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

In [ ]:
df2

In [ ]:
def dtm_todf(dtm, vect):
    return pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

In [ ]:
dtm_todf(dtm, vect)

__EXERCISE:__

  * 1) Transform new_str below into a document term matrix with our fitted CountVectorizer. Does the output look like you thought?

In [ ]:
new_str = ['kitten puPPy fish', "Fish dog's"]

In [ ]:
# %load exercise1.txt

<a id='1.1.3'></a>
### 1.1.3 Parameters

This class has many parameters but we will highlight a few of the most important ones

<a id='1.1.3.1'></a>
#### 1.1.3.1 min_df

__min_df__ : float in range [0.0, 1.0] or int, default=1

When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

__Example__

We will show what the document term matrix looks like before and after min_df parameter is used

In [ ]:
cv = CountVectorizer()

In [ ]:
train_dtm = cv.fit_transform(df['Text'])

In [ ]:
train_dtm.shape

So the shape of our dtm is (2833, 38850) before using min_df.

Lets see what the shape is after using a min_df of 5

In [ ]:
cv = CountVectorizer(min_df=5)

In [ ]:
train_dtm = cv.fit_transform(df['Text'])

In [ ]:
train_dtm.shape

Too see how the min_df affects the dtm's shape for different values we will number of columns below for a min_df from 1 to 10

In [ ]:
start = 1
end = 10
plt.figure(figsize=(8,8))
plt.title('Columns vs min_df')
plt.xlabel('min_df')
plt.ylabel('Column Number')
for min_df in np.arange(start,end,1):
    cv = CountVectorizer(min_df=min_df)
    train_dtm = cv.fit_transform(df['Text'])
    num_cols = train_dtm.shape[1]
    print('min_df={} has shape {}'.format(min_df, train_dtm.shape))
    plt.scatter(min_df, num_cols)

<a id='1.1.3.2'></a>
#### 1.1.3.2 max_df

__max_df :__ float in range [0.0, 1.0] or int, default=1.0

When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

__Example__

We will show what max_df does to the document term matrix

In [ ]:
cv = CountVectorizer()
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with default max_df: {}".format(train_dtm.shape))

In [ ]:
cv = CountVectorizer(max_df=.5)
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with .5 max_df: {}".format(train_dtm.shape))

To view the impact of max_df for different values we will plot the feature space for a range of max_df values

In [ ]:
plt.figure(figsize=(8,8))
plt.ylabel('Number of Columns')
plt.xlabel('Value of max_df')
plt.title('Columns vs max_df')
for max_df in np.arange(.1,1.0,.1):
    cv = CountVectorizer(max_df=max_df)
    train_dtm = cv.fit_transform(df['Text'])
    num_cols = train_dtm.shape[1]
    print('max_df={:f} has shape {}'.format(max_df, train_dtm.shape))
    plt.scatter(max_df, num_cols)

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Columns vs max_df')
plt.xlabel('max_df')
plt.ylabel('Number Columns')
for max_df in np.arange(25,200,25):
    cv = CountVectorizer(max_df=max_df)
    train_dtm = cv.fit_transform(df['Text'])
    num_cols = train_dtm.shape[1]
    print('max_df={} has shape {}'.format(max_df, train_dtm.shape))
    plt.scatter(max_df, num_cols)

__Exercise:__

  * 1) Find a max_df value that will return half the number of columns (19,000)
  * 2) Find a min_df value that will return half the number of columns

In [ ]:
# %load exercise2.txt

<a id='1.1.3.3'></a>
#### 1.1.3.3 ngram_range

__ngram_range :__ tuple (min_n, max_n)

The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used.

__Example__

We will show what happens to the document term matrix when different combinations are used

In [ ]:
cv = CountVectorizer(ngram_range=(1,1))
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with ngram_range=(1,1): {}".format(train_dtm.shape))

In [ ]:
cv = CountVectorizer(ngram_range=(1,2))
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with ngram_range=(1,2): {}".format(train_dtm.shape))

In [ ]:
text = ['cat dog dog dog dog dog dog dog', 'cat fish puppy', 'fish cat']
cv = CountVectorizer(ngram_range=(1,2))
test_dtm = cv.fit_transform(text)
dtm_todf(test_dtm, cv)

<a id='1.1.3.4'></a>
#### 1.1.3.4 stop_words

__stop_words :__ string {‘english’}, list, or None (default)

If ‘english’, a built-in stop word list for English is used.
If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens. Only applies if analyzer == 'word'.

If None, no stop words will be used. max_df can be set to a value in the range [0.7, 1.0) to automatically detect and filter stop words based on intra corpus document frequency of terms.

__Example__

We will show how removing stop_words affects our document term matrix

In [ ]:
cv = CountVectorizer(stop_words=None)
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with stop_words=None: {}".format(train_dtm.shape))

In [ ]:
cv = CountVectorizer(stop_words='english')
train_dtm = cv.fit_transform(df['Text'])
print("The shape of the dtm with stop_words=None: {}".format(train_dtm.shape))

<a id='1.1.3.5'></a>
#### 1.1.3.5 max_features

__max_features :__ int or None, default=None

If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

This parameter is ignored if vocabulary is not None.

__Example__

We will show how max_features affects our dtm, and how it interacts with other CountVectorizer parameters

In [ ]:
cv = CountVectorizer(max_features=20)
train_dtm = cv.fit_transform(df['Text'])
dtm_todf(train_dtm, cv).head()

In [ ]:
cv = CountVectorizer(max_features=20, stop_words='english')
train_dtm = cv.fit_transform(df['Text'])
dtm_todf(train_dtm, cv).head()

How can we get rid of these?

__Exercise:__

  * 1) Find the 20 most common bi-grams in our dataset

In [ ]:
#%load exercise3.txt

<a id='1.2'></a>
## 1.2 TfidfTransformer

Counting occurences is a good start but there is still the issue of longer documents having higher average count values than shorter documents, even though they might talk about the same topics.

The way to get around this issue is to divide the number of occurences of each word in each document by the total number of words in the document. These new features are called term frequencies.

Another common addition to term frequencies is to downscale weights for words that will occur in many documents in the corpus. This will help us in our models because we are making the assumption that these common words are less informative than the words that are less frequent.

This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

This process is implemented by the __TfidfTransformer__ in sklearn, to read more about it click [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)

<a id='1.2.1'></a>
### 1.2.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)__

Transform a count matrix to a normalized tf or tf-idf representation

Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

The formula that is used to compute the tf-idf of term t is tf-idf(d, t) = tf(t) * idf(d, t), and the idf is computed as idf(d, t) = log [ n / df(d, t) ] + 1 (if smooth_idf=False), where n is the total number of documents and df(d, t) is the document frequency; the document frequency is the number of documents d that contain term t. The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored. (Note that the idf formula above differs from the standard textbook notation that defines the idf as idf(d, t) = log [ n / (df(d, t) + 1) ]).

If smooth_idf=True (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(d, t) = log [ (1 + n) / 1 + df(d, t) ] + 1.

Furthermore, the formulas used to compute tf and idf depend on parameter settings that correspond to the SMART notation used in IR as follows:

Tf is “n” (natural) by default, “l” (logarithmic) when sublinear_tf=True. Idf is “t” when use_idf is given, “n” (none) otherwise. Normalization is “c” (cosine) when norm='l2', “n” (none) when norm=None.

<a id='1.2.2'></a>
### 1.2.2 Example

To get a better understanding of the class we will do an example below

In [ ]:
text = ['cat dog dog dog dog dog dog dog', 'cat fish puppy', 'fish cat']
vect = CountVectorizer()
dtm = vect.fit_transform(text)
example_df = dtm_todf(dtm, vect)

In [ ]:
example_df.head()

In [ ]:
tfidf = TfidfTransformer()

In [ ]:
dtm = tfidf.fit_transform(example_df)

In [ ]:
dtm_todf(dtm, vect)

These results are hard to explain without first looking at each default parameter individually

<a id='1.2.3'></a>
### 1.2.3 Parameters

This class has much less parameters than CountVectorizer but these parameters aren't as intuitive so we'll cover them below

<a id='1.2.3.1'></a>
#### 1.2.3.1 norm

__norm :__ ‘l1’, ‘l2’ or None, optional

Norm used to normalize term vectors. None for no normalization.

__Example__

We will compare what each norm does to our document term matrix

In [ ]:
example_df.head()

We haven't covered it yet but the use_idf parameter can be set to false so only term frequencies are done. We will set it equal to false for this example because it will really help to highlight what the norm parameter is doing

__norm = 'l1'__

In [ ]:
tfidf = TfidfTransformer(norm='l1', use_idf=False)

In [ ]:
dtm = tfidf.fit_transform(example_df)

In [ ]:
dtm_todf(dtm, vect)

__norm = 'l2'__

In [ ]:
tfidf = TfidfTransformer(norm='l2', use_idf=False)

In [ ]:
dtm = tfidf.fit_transform(example_df)

In [ ]:
dtm_todf(dtm, vect)

What's the l2 norm doing to our document term matrix?

In [ ]:
mat = example_df.as_matrix()

In [ ]:
[np.sqrt(x**2/sum(x**2)) for x in mat]

__norm = None__

This is perhaps the least exciting norm, but it is still very important

In [ ]:
tfidf = TfidfTransformer(norm=None, use_idf=False)

In [ ]:
dtm = tfidf.fit_transform(example_df)

In [ ]:
dtm_todf(dtm, vect)

<a id='1.2.3.2'></a>
#### 1.2.3.2 use_idf

__use_idf :__ boolean, default=True

Enable inverse-document-frequency reweighting.

__Example__

We will show how use_idf=True will interact with the different norms

In [ ]:
norm_options = ['l1', 'l2', 'None']
@interact
def try_norm(norm=norm_options):
    if norm == 'None':
        norm = None
    tfidf = TfidfTransformer(norm=norm, use_idf=True)
    dtm = tfidf.fit_transform(example_df)
    print(dtm_todf(dtm, vect))

<a id='2'></a>
# 2 Modeling

Now that we learned the basics we can begin the modeling process. There are many different algorithms and techniques but I will only cover two of the ones we tried in our practicum project.

We will also split our data up here:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Target'], test_size=.2, random_state=0)

<a id='2.1'></a>
## 2.1 MultinomialNB

Multinomial Naive Bayes is known as a good dependable baseline for text classification

__Advantages:__ 

  * Very fast
  * Robust to irrelevant features
  * Works well with little data
  * Very hard to overfit due to simplicity

__Disadvantages:__

  * Will most likely underfit if independence assumption doesn't hold

<a id='2.1.1'></a>
### 2.1.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)__

Naive Bayes classifier for multinomial models

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

<a id='2.1.2'></a>
### 2.1.2 Example

We will begin the modeling process with the actual dataset now

We will just use the default parameters and see what they give us.

__NOTE:__ Make sure you're not fitting CountVectorizer or TfidfTransformer with test data! This is cheating and no one likes a cheater!

__FITTING TRAIN SET__

In [ ]:
cv = CountVectorizer()
clf = MultinomialNB()

X_train_dtm = cv.fit_transform(X_train)
clf.fit(X_train_dtm, y_train)

__PREDICTING TEST SET__

In [ ]:
X_test_dtm = cv.transform(X_test)
y_pred = clf.predict(X_test_dtm)

__EVALUATING PREDICTIONS__

In [ ]:
def evaluate_model(actual, prediction):
    
    print("Classification report: \n{}".format(classification_report(actual, prediction)))
    print('Confusion Matrix: \n{}'.format(confusion_matrix(actual, prediction)))
    print('\nAccuracy: {}'.format(accuracy_score(actual, prediction)))

In [ ]:
evaluate_model(y_test, y_pred)

In [ ]:
idf_options = ['False', 'True']
norm_options = ['l1', 'l2', 'None']
@interact
def try_alpha(use_idf=idf_options, norm=norm_options):
    clf = MultinomialNB()
    
    if norm == 'None':
        norm = None
    if use_idf == 'False':
        use_idf = False
    else:
        use_idf = True
    tfidf = TfidfTransformer(use_idf=use_idf, norm=norm)
    
    new_train_dtm = tfidf.fit_transform(X_train_dtm)
    new_test_dtm = tfidf.transform(X_test_dtm)
    
    clf.fit(new_train_dtm, y_train)
    y_pred = clf.predict(new_test_dtm)
    evaluate_model(y_test, y_pred)

<a id='2.1.3'></a>
### 2.1.3 Parameters

One of the benefits of multinomial naive bayes is that there are very little parameters to tune

<a id='2.1.3.1'></a>
#### 2.1.3.1 alpha

__alpha :__ float, optional (default=1.0)

Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).

__Example__

We will show what a different alpha value does to coefficients and prediction power


In [ ]:
cv = CountVectorizer(max_features=1000)
X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

In [ ]:
alphas = [0, .00001, .0001, .0001, .001, .01, .1, 1]
idf_options = [False, True]
norm_options = ['l1', 'l2', None]
@interact
def try_alpha(alpha=alphas):
    clf = MultinomialNB(alpha=float(alpha))
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    evaluate_model(y_test, y_pred)
    

<a id='2.2'></a>
## 2.2 SGDClassifier

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier):__

Linear classifiers (SVM, logistic regression, a.o.) with SGD training.

This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning, see the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

This implementation works with data represented as dense or sparse arrays of floating point values for the features. The model it fits can be controlled with the loss parameter; by default, it fits a linear support vector machine (SVM).

The regularizer is a penalty added to the loss function that shrinks model parameters towards the zero vector using either the squared euclidean norm L2 or the absolute norm L1 or a combination of both (Elastic Net). If the parameter update crosses the 0.0 value because of the regularizer, the update is truncated to 0.0 to allow for learning sparse models and achieve online feature selection.


<a id='2.2.1'></a>
### 2.2.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/sgd.html):__

Stochastic Gradient Descent (SGD) is a simple yet very efficient approach to discriminative learning of linear classifiers under convex loss functions such as (linear) Support Vector Machines and Logistic Regression. Even though SGD has been around in the machine learning community for a long time, it has received a considerable amount of attention just recently in the context of large-scale learning.

SGD has been successfully applied to large-scale and sparse machine learning problems often encountered in text classification and natural language processing. Given that the data is sparse, the classifiers in this module easily scale to problems with more than 10^5 training examples and more than 10^5 features.

The __advantages__ of Stochastic Gradient Descent are:

  * Efficiency.
  * Ease of implementation (lots of opportunities for code tuning).

The __disadvantages__ of Stochastic Gradient Descent include:

  * SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.
  * SGD is sensitive to feature scaling.

<a id='2.2.2'></a>
### 2.2.2 Example

We will try SGD with the default parameters

__FITTING THE TRAINING SET__

In [ ]:
cv = CountVectorizer()
clf = SGDClassifier(random_state=0)

X_train_dtm = cv.fit_transform(X_train)
clf.fit(X_train_dtm, y_train)

__PREDICTING TEST__

In [ ]:
X_test_dtm = cv.transform(X_test)
y_pred = clf.predict(X_test_dtm)

__EVALUATING MODEL__

In [ ]:
evaluate_model(y_test, y_pred)

Interesting, why didn't this model perform better than multinomial naive bayes?

__EXERCISE:__

  * 1) Re-run the model with TfidfTransformer. Are the results better?

In [ ]:
#%load exercise4.txt

<a id='2.2.3'></a>
### 2.2.3 Parameters

SGDClassifier has many parameters, we'll examine a few of the more important ones

<a id='2.2.3.1'></a>
#### 2.2.3.1 loss

__loss :__ str, ‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’

The loss function to be used. Defaults to ‘hinge’, which gives a linear SVM. The ‘log’ loss gives logistic regression, a probabilistic classifier. ‘modified_huber’ is another smooth loss that brings tolerance to outliers as well as probability estimates. ‘squared_hinge’ is like hinge but is quadratically penalized. ‘perceptron’ is the linear loss used by the perceptron algorithm. The other losses are designed for regression but can be useful in classification as well; see SGDRegressor for a description.

__Example__

We will try a few different loss functions to see their effect

In [ ]:
loss_options = ['log', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron']
cv = CountVectorizer()
X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

tfidf = TfidfTransformer()
X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)
@interact
def try_loss_functions(loss=loss_options):
    
    clf = SGDClassifier(loss=loss, random_state=0)
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    evaluate_model(y_test, y_pred)

<a id='2.2.3.2'></a>
#### 2.2.3.2 penalty

__penalty :__ str, ‘none’, ‘l2’, ‘l1’, or ‘elasticnet’

The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. ‘l1’ and ‘elasticnet’ might bring sparsity to the model (feature selection) not achievable with ‘l2’.

In [ ]:
penalty_options = ['l1', 'l2', 'elasticnet']
alpha_options = [.000001, .00001, .0001, .001]
cv = CountVectorizer()
X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

tfidf = TfidfTransformer()
X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)
@interact
def try_penalties(penalty=penalty_options, alpha=alpha_options):
    
    clf = SGDClassifier(penalty=penalty, alpha=float(alpha), random_state=0)
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    evaluate_model(y_test, y_pred)

<a id='2.2.3.3'></a>
#### 2.2.3.3 alpha

__alpha :__ float

Constant that multiplies the regularization term. Defaults to 0.0001 Also used to compute learning_rate when set to ‘optimal’.

__Example__

We will plot a range of alpha values to see how they affect our test sample's accuracy

In [ ]:
# values = []
cv = CountVectorizer()
tfidf = TfidfTransformer()

X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)
X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)

for i, alpha in enumerate(list(np.logspace(-6, -3))):
    
    clf = SGDClassifier(random_state=0, alpha=alpha, penalty='l1')
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    plt.scatter(i, accuracy_score(y_test, y_pred))
    plt.scatter(i, accuracy_score(y_train, clf.predict(X_train_dtm)), color='red')

<a id='2.2.3.4'></a>
#### 2.2.3.4 learning_rate

__learning_rate :__ string, optional

The learning rate schedule:
  * ‘constant’: eta = eta0
  * ‘optimal’: eta = 1.0 / (alpha * (t + t0)) [default]
  * ‘invscaling’: eta = eta0 / pow(t, power_t)
  
where t0 is chosen by a heuristic proposed by Leon Bottou.

__Example__

In [ ]:
learning_options = ['constant', 'optimal', 'invscaling']
eta_options = [.0001, .001, .01, .1]
cv = CountVectorizer()
X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

tfidf = TfidfTransformer()
X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)
@interact
def try_epsilon(learning_rate=learning_options, eta0=eta_options):
    
    clf = SGDClassifier(learning_rate=learning_rate, eta0=float(eta0), random_state=0)
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    evaluate_model(y_test, y_pred)

<a id='2.2.4'></a>
### 2.2.4 Getting best features out

SGDClassifier makes it very easy to get the best features out of our model, which can be very helpful if we're trying to shrink our feature space

In [ ]:
#Simple model fit for SGDClassifier
#----------------------------------
cv = CountVectorizer()
tfidf = TfidfTransformer()

X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)

clf.fit(X_train_dtm, y_train)

In [ ]:
def get_coef_df(sgd_clf, vect):
    
    sgd_clf.densify()
    ndf = pd.DataFrame(clf.coef_.T, columns=train.target_names)
    ndf['Token'] = cv.get_feature_names()
    return ndf

In [ ]:
coef_df = get_coef_df(clf, cv)

In [ ]:
coef_df.head()

In [ ]:
def get_most_important(ratio_df, num, return_tokens=False, ascending=False):
    
    list_tokens = []
    for column in ratio_df.columns[:-1]:
        ratio_df = ratio_df.sort_values(by=column, ascending=ascending)
        list_tokens += ratio_df.iloc[:num]['Token'].tolist()
        if return_tokens == False:
            print("Token coefficients for {}:\n".format(column), ratio_df[['Token', column]][:num])
    if return_tokens == True:
        return list(set(list_tokens))   

In [ ]:
get_most_important(coef_df, 5)

__Exercise:__

  * 1) Get the 50 most important bi-grams

In [ ]:
# %load exercise5.txt

In [ ]:
def plot_important_features(coef, feature_names, top_n=20, ax=None):
    if ax is None:
        ax = plt.gca()
    inds = np.argsort(coef)
    low = inds[:top_n]
    high = inds[-top_n:]
    important = np.hstack([low, high])
    myrange = range(len(important))

    ax.bar(myrange, coef[important])
    ax.set_xticks(myrange)
    ax.set_xticklabels(feature_names[important], rotation=60, ha="right")

In [ ]:
names = pd.Series(cv.get_feature_names())
plt.figure(figsize=(12,8))
plot_important_features(coef_df['talk.politics.misc'], names)

In [ ]:
top_words = get_most_important(coef_df, 100, return_tokens=True, ascending=True)
bottom_words = get_most_important(coef_df, 100, return_tokens=True, ascending=False)

In [ ]:
best_words = top_words + bottom_words

In [ ]:
len(best_words)

In [ ]:
len(set(best_words))

In [ ]:
cv = CountVectorizer(vocabulary=set(top_words))
clf = SGDClassifier(random_state=0)
tfidf = TfidfTransformer()

X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)

clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
evaluate_model(y_test, y_pred)

In [ ]:
cv = CountVectorizer(stop_words='english', max_df=.4, max_features=700)
clf = SGDClassifier(random_state=0)
tfidf = TfidfTransformer()

X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

X_train_dtm = tfidf.fit_transform(X_train_dtm)
X_test_dtm = tfidf.transform(X_test_dtm)

clf.fit(X_train_dtm, y_train)
y_pred = clf.predict(X_test_dtm)
evaluate_model(y_test, y_pred)

In [ ]:
results = []
start = 50#50
end = 400#200
intervals = np.linspace(start, end, num=10)
for num in [int(x) for x in intervals]:
    
    best_tokens = get_most_important(coef_df, num, return_tokens=True, ascending=False)
    best_tokens += get_most_important(coef_df, num, return_tokens=True, ascending=True)
    
    best_cv = CountVectorizer(vocabulary=set(best_tokens), ngram_range=(1,1))
    best_tfidf = TfidfTransformer()
    
    best_train_dtm = best_cv.fit_transform(X_train)
    
    best_nb = MultinomialNB()
    best_nb.fit(best_train_dtm, y_train)
    best_nb_acc = accuracy_score(y_test, best_nb.predict(best_cv.transform(X_test)))
    
    best_sgd = SGDClassifier(random_state=0)
    best_train_dtm = best_tfidf.fit_transform(best_train_dtm)
    best_test_dtm = best_tfidf.transform(best_cv.transform(X_test))
    best_sgd.fit(best_train_dtm, y_train)
    
    best_sgd_acc = accuracy_score(y_test, best_sgd.predict(best_test_dtm))
                                 
    
    max_cv = CountVectorizer(max_features=len(set(best_tokens)), ngram_range=(1,1), stop_words='english', max_df=.2)
    max_tfidf = TfidfTransformer()
    max_train_dtm = max_cv.fit_transform(X_train)
    
    max_nb = MultinomialNB()
    max_nb.fit(max_train_dtm, y_train)
    max_nb_acc = accuracy_score(y_test, max_nb.predict(max_cv.transform(X_test)))
                                  
    max_sgd = SGDClassifier(random_state=0)
    max_train_dtm = max_tfidf.fit_transform(max_train_dtm)
    max_test_dtm = max_tfidf.transform(max_cv.transform(X_test))
    max_sgd.fit(max_train_dtm, y_train)
    max_sgd_acc = accuracy_score(y_test, max_sgd.predict(max_cv.transform(X_test)))
    
    results.append((len(set(best_tokens)), best_nb_acc, best_sgd_acc, max_nb_acc, max_sgd_acc))                            

In [ ]:
results_df = pd.DataFrame(results, columns=['Num_Features', 'Best_NB', 'Best_SGD', 'Max_NB', 'Max_SGD'])

In [ ]:
plt.figure(figsize=(8,8))
results_df.set_index('Num_Features').plot()

<a id='3'></a>
# 3 Tuning Models

The models default parameters give us a good model, but if we want a great model we'll have to start turning the knobs and find our own optimal parameters

<a id='3.1'></a>
## 3.1 Pipeline

Typing new parameters and keeping track of their results is a pain, lucky for us sklearn has a wonderful class called pipeline so we don't have to do that.

<a id='3.1.1'></a>
### 3.1.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html):__

Pipeline of transforms with a final estimator.

Sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.

The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters. For this, it enables setting parameters of the various steps using their names and the parameter name separated by a ‘\_\_’, as in the example below. A step’s estimator may be replaced entirely by setting the parameter with its name to another estimator, or a transformer removed by setting to None.

<a id='3.1.2'></a>
### 3.1.2 Example

__1.__ We'll create a simple pipeline for SGDClassifier using the [make_pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) function in sklearn

In [ ]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer(), SGDClassifier(random_state=0))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
evaluate_model(y_test, y_pred)

__2.__ We'll now build a pipeline without the make_pipeline helper function

In [ ]:
pipeline = Pipeline([
        ('cv', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('sgd', SGDClassifier(random_state=0))
    ])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
evaluate_model(y_test, y_pred)

<a id='3.2'></a>
## 3.2 Parameter Search

The main reason these pipelines are used is to help people with parameter searching. There are two popular parameter search classes in sklearn

<a id='3.2.1'></a>
### 3.2.1 GridSearchCV

This is most well known searching technique in sklearn

<a id='3.2.1.1'></a>
#### 3.2.1.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html):__

Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

<a id='3.2.1.2'></a>
#### 3.2.1.2 Example

We'll show a small grid search for the optimal parameters for our multinomial naive bayes model

In [ ]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer(), MultinomialNB())

#These are the parameters we're searching over
params = {
    'countvectorizer__ngram_range':[(1,1), (1,2)],
    #'countvectorizer__min_df':[2, 3, 4, 5],
    #'countvectorizer__max_df':[.2, .4, .6, .8, 1.0],
    'tfidftransformer__norm':['l2', None],
    'tfidftransformer__use_idf':[True, False],
    #'multinomialnb__alpha':[.001, .01, .1, 1]
}
grid = GridSearchCV(pipe, params, cv=5, verbose=True, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
def evaluate_grid(_grid, metric='accurracy'):
    print("Best CV Score: {:.2f}".format(_grid.best_score_))
    print("Best parameters: {}".format(_grid.best_params_))
    y_pred = _grid.predict(X_test)
    print("\taccuracy score on test: {}".format(accuracy_score(y_test, y_pred)))
    print("Classification report for test sample held out: \n{}".format(classification_report(y_test, y_pred)))
    print("Confusion Matrix for test: \n{}".format(confusion_matrix(y_test, y_pred)))
    

Scores for each cross-validation are saved inside GridSearchCV object

In [ ]:
results = pd.DataFrame(grid.cv_results_).fillna('None')

In [ ]:
res_pivot = results.pivot_table(values=['mean_test_score', 'mean_train_score'],
                                index=["param_countvectorizer__ngram_range", "param_tfidftransformer__use_idf",
                                       "param_tfidftransformer__norm"])

In [ ]:
res_pivot.mean_test_score

<a id='3.2.1.3'></a>
#### 3.2.1.3 Parameters

GridSearchCV has some very important parameters that will save you time

__scoring :__ string, callable or None, default=None

A string (see model evaluation documentation) or a scorer callable object / function with signature scorer(estimator, X, y). If None, the score method of the estimator is used.

__[LIST OF SCORING FUNCTIONS](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter):__

<img src="./sklearn_scoring_table.png">

In [ ]:
X_train.head()

In [ ]:
politics = y_train[y_train == 4]

In [ ]:
len(politics)

In [ ]:
politics.index

In [ ]:
X_sample_df = X_train.drop(politics.index[:300])
y_sample_df = y_train.drop(politics.index[:300])

In [ ]:
len(X_train)

In [ ]:
len(X_sample_df)

In [ ]:
Counter(y_sample_df)

In [ ]:
sample_X_train, sample_X_test, sample_y_train, sample_y_test = train_test_split(X_sample_df,
                                                                                y_sample_df, test_size=.2, random_state=0)

In [ ]:
scoring = ['accuracy', 'neg_log_loss', 'f1_macro']

@interact
def try_score(score=scoring):
    pipe = make_pipeline(CountVectorizer(max_features=2000), TfidfTransformer(), MultinomialNB())

    #These are the parameters we're searching over
    params = {
        #'countvectorizer__ngram_range':[(1,1), (1,2)],
        #'countvectorizer__min_df':[2, 3, 4, 5],
        #'countvectorizer__max_df':[.2, .4, .6, .8, 1.0],
        'tfidftransformer__norm':['l1', 'l2', None],
        'tfidftransformer__use_idf':[True, False],
        'multinomialnb__alpha':[.001, .01, .1, 1]
    }
    grid = GridSearchCV(pipe, params, cv=3, verbose=True, n_jobs=-1, scoring=score)
    grid.fit(sample_X_train, sample_y_train)
    
    
    print("Best CV Score: {:.2f}".format(grid.best_score_))
    print("Best parameters: {}".format(grid.best_params_))
    sample_y_pred = grid.predict(sample_X_test)
    print("\taccuracy score on test: {}".format(accuracy_score(sample_y_test, sample_y_pred)))
    print("Classification report for test sample held out: \n{}".format(classification_report(sample_y_test, sample_y_pred)))
    print("Confusion Matrix for test: \n{}".format(confusion_matrix(sample_y_test, sample_y_pred)))
    #evaluate_grid(grid)

__n_jobs :__ int, default=1

Number of jobs to run in parallel.

In [ ]:
t0 = time()
n_jobs = 1
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

#These are the parameters we're searching over
params = {
    'countvectorizer__ngram_range':[(1,1), (1,2)],
    'countvectorizer__stop_words':['english', None],
    #'countvectorizer__min_df':[2, 3, 4, 5],
    #'countvectorizer__max_df':[.2, .4, .6, .8, 1.0],
}
grid = GridSearchCV(pipe, params, cv=5, verbose=True, n_jobs=n_jobs)
grid.fit(X_train, y_train)
print('Took {} seconds with n_jobs={}'.format(time()-t0, n_jobs))

In [ ]:
t0 = time()
n_jobs = -1
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

#These are the parameters we're searching over
params = {
    'countvectorizer__ngram_range':[(1,1), (1,2)],
    'countvectorizer__stop_words':['english', None],
    #'countvectorizer__min_df':[2, 3, 4, 5],
    #'countvectorizer__max_df':[.2, .4, .6, .8, 1.0],
}
grid = GridSearchCV(pipe, params, cv=5, verbose=True, n_jobs=n_jobs)
grid.fit(X_train, y_train)
print('Took {} seconds with n_jobs={}'.format(time()-t0, n_jobs))

__cv :__ int, cross-validation generator or an iterable, optional

Determines the cross-validation splitting strategy. Possible inputs for cv are:

  * None, to use the default 3-fold cross validation,

  * integer, to specify the number of folds in a (Stratified)KFold,

  * An object to be used as a cross-validation generator.

  * An iterable yielding train, test splits.

For integer/None inputs, if the estimator is a classifier and y is either binary or multiclass, [StratifiedKFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold) is used. In all other cases, KFold is used.

Refer [User Guide](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) for the various cross-validation strategies that can be used here.

In [ ]:
from sklearn.model_selection import ShuffleSplit

shufflesplit = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

#These are the parameters we're searching over
params = {
    'countvectorizer__ngram_range':[(1,1), (1,2)],
    #'countvectorizer__min_df':[2, 3, 4, 5],
    #'countvectorizer__max_df':[.2, .4, .6, .8, 1.0],
    #'multinomialnb__alpha':[.001, .01, .1, 1]
}
grid = GridSearchCV(pipe, params, cv=shufflesplit, verbose=True, n_jobs=-1)
grid.fit(X_train, y_train)
evaluate_grid(grid)

<a id='3.2.2'></a>
### 3.2.2 RandomizedSearchCV

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-search):__

While using a grid of parameter settings is currently the most widely used method for parameter optimization, other search methods have more favourable properties. RandomizedSearchCV implements a randomized search over parameters, where each setting is sampled from a distribution over possible parameter values. This has two main benefits over an exhaustive search:

  * A budget can be chosen independent of the number of parameters and possible values.
  * Adding parameters that do not influence the performance does not decrease efficiency.
  
Specifying how parameters should be sampled is done using a dictionary, very similar to specifying parameters for GridSearchCV.

Additionally, a computation budget, being the number of sampled candidates or sampling iterations, is specified using the n_iter parameter. For each parameter, either a distribution over possible values or a list of discrete choices (which will be sampled uniformly) can be specified:

<a id='3.2.2.1'></a>
#### 3.2.2.1 Description

__FROM SKLEARN [DOCUMENTATION](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html):__

Randomized search on hyper parameters.

RandomizedSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated search over parameter settings.
In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.

If all parameters are presented as a list, sampling without replacement is performed. If at least one parameter is given as a distribution, sampling with replacement is used. It is highly recommended to use continuous distributions for continuous parameters.

<a id='3.2.2.2'></a>
#### 3.2.2.2 Example

RandomizedSearchCV is very similar to GridSearchCV with the only difference being n_iter parameter which will allow us to select from a bigger parameter grid

In [ ]:
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

#These are the parameters we're searching over
params = {
    'countvectorizer__ngram_range':[(1,1), (1,2), (1,3)],
    'countvectorizer__min_df':[0,2,3,4,5,6,7,8,9,10],
    'countvectorizer__max_df':[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0],
    'multinomialnb__alpha':[.00001, .0001, .001, .01, .1, 1]
}
grid = RandomizedSearchCV(pipe, params, cv=5, verbose=True, n_iter=10, n_jobs=-1)
grid.fit(X_train, y_train)
evaluate_grid(grid)

__EXERCISE:__

  * 1) Create a pipeline and Grid/Randomized search CV for optimal parameters for SGDClassifier

<a id='4'></a>
# 4 Increasing Performance

There are many ways to increase model performance. The best way is typically feature engineering, but we're just covering text mining here

<a id='4.1'></a>
## 4.1 Stemming

One thing we can do is stem our text to shrink our feature space. This sometimes gives a better model but not always, sometimes the different classes with use different suffixes

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
import nltk

<a id='4.1.1'></a>
### 4.1.1 Types

We will only focus on three of the more popular stemmers

### Porter

This is the most common stemmer

__EXAMPLE__

In [ ]:
p_stemmer = PorterStemmer()

In [ ]:
p_stemmer.stem('player')

### Lancaster

This is a more aggresive stemmer, meaning it will chop off suffixes more frequently

__EXAMPLE__

In [ ]:
l_stemmer = LancasterStemmer()

In [ ]:
l_stemmer.stem('player')

### Snowball 

Adapted version of Porter, must specify the language

__EXAMPLE__

In [ ]:
s_stemmer = SnowballStemmer('english')

In [ ]:
s_stemmer.stem('generously')

In [ ]:
p_stemmer.stem('generously')

In [ ]:
l_stemmer.stem('generously')

<a id='4.1.2'></a>
### 4.1.2 Stemming our Text

Before we begin stemming our words we need to make sure stop_words aren't being stemmed because they won't be picked up by our CountVectorizer if the word is different.

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

In [ ]:
stop_words[:10]

In [ ]:
for word in stop_words[50:100]:
    
    if len(word) > 5:
        print('\nStop word: {}'.format(word))
        print('\tPorter: {}\n\tLancaster: {}\n\tSnowball: {}'.format(p_stemmer.stem(word), l_stemmer.stem(word),
                                                                          s_stemmer.stem(word)))

We will then have to ignore these words when stemming

In [ ]:
token_regex = re.compile(r'\b\w\w+\b')
def stem_column(col, stemmer, stop_words):
    return col.apply(lambda x: ' '.join([stemmer.stem(token) if token not in stop_words else token
                                         for token in token_regex.findall(x)]))

In [ ]:
test_df = df['Text'].head(10)
snowball_text = stem_column(test_df, s_stemmer, stop_words)
porter_text = stem_column(test_df, p_stemmer, stop_words)
lancaster_text = stem_column(test_df, l_stemmer, stop_words)

In [ ]:
i = 4
print('Original: \n{}'.format(test_df.iloc[i]))
print('\nSnowball Stemmed: \n\n{}'.format(snowball_text.iloc[i]))
print('\nPorter Stemmed: \n\n{}'.format(snowball_text.iloc[i]))
print('\nLancaster Stemmed: \n\n{}'.format(snowball_text.iloc[i]))

In [ ]:
df['snowball_text'] = stem_column(df['Text'], s_stemmer, stop_words)
df['porter_text'] = porter_text = stem_column(df['Text'], p_stemmer, stop_words)
df['lancaster_text'] = lancaster_text = stem_column(df['Text'], l_stemmer, stop_words)

In [ ]:
for col in ['Text', 'snowball_text', 'porter_text', 'lancaster_text']:
    cv = CountVectorizer(stop_words='english')
    dtm = cv.fit_transform(df[col])
    print('There are {} tokens for column: {}'.format(dtm.shape[1], col))

<a id='4.2'></a>
## 4.2 Custom Transformer (Advanced)

We know pipelines take in transformers, but until now we've been using sklearns transformers. Most of the time this will be fine, but if you know how to create your own transformer you'll have much greater power.

I already created one and saved it in file column_selector.py, this is necessary for GridSearchCV

In [ ]:
from column_selector import ColSelector

In [ ]:
??ColSelector

This is a very simple transformer, that just grabs what ever column from the data set you specify

In [ ]:
col = ColSelector(key='Text')

In [ ]:
col.fit_transform(df)

In [ ]:
pipeline = Pipeline([
    ('selector', ColSelector(key='Text')),
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier())
    ])

params = {
    'selector__key':['Text', 'snowball_text', 'porter_text', 'lancaster_text'],
    'cv__stop_words':['english'],
    'cv__ngram_range':[(1,1), (1,2), (1,3)],
    'cv__min_df':[2, 3, 4],
    'cv__max_df':[.5, .6, .7, .8, .9, 1.0],
    'tfidf__use_idf':[True],
    'tfidf__norm':['l1', 'l2'],
    'sgd__alpha':np.logspace(-7, -2, 20),
    'sgd__loss':['hinge', 'modified_huber'],
    'sgd__epsilon':np.logspace(-6, 0, 10)
}

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df, train.target, test_size=.2, random_state=0)
n_iter = 200
t0 = time()

stemming_grid = RandomizedSearchCV(pipeline, params, cv=5, n_iter=n_iter, n_jobs=-1, verbose=True)
stemming_grid.fit(X_train2, y_train2)

In [ ]:
print("Best CV Score: {:.2f}".format(stemming_grid.best_score_))
print("Best parameters: {}".format(stemming_grid.best_params_))
y_pred = stemming_grid.predict(X_test2)
print("Classification report for test sample held out: \n{}".format(classification_report(y_test2, y_pred)))
print("Confusion Matrix for test: \n{}".format(confusion_matrix(y_test2, y_pred)))
print('Ran {} iterations in {} seconds'.format(n_iter, time()-t0))

In [ ]:
results = pd.DataFrame(stemming_grid.cv_results_)

In [ ]:
results.head(10)

In [ ]:
results.groupby(by='param_selector__key')['mean_test_score'].mean()

In [ ]:
results = results.fillna('None')

In [ ]:
param_cols = [col for col in results.columns if col.startswith('param_')]
@interact
def compare_parameter(col=param_cols):
    print(results.groupby(by=col)['mean_test_score'].mean())

__EXERCISE__

  * 1) Write a text classification pipeline to classify yelp reviews between 1-5 stars.
  * 2) Find a good set of parameters using grid search.
  * 3) Evaluate the performance on a held out test set.
  
  __Extra:__ Find a new sklearn classifier and build a pipeline with its parameters

In [ ]:
df = pd.read_csv('./data/yelp.csv')